In [1]:
import s3fs
import xarray as xr

In [ ]:
# 1. Point s3fs at the public bucket (no credentials needed)
fs = s3fs.S3FileSystem(anon=True)
date = 20250603
# 2. List a directory (e.g. daily time_series files for soil moisture)
path = f"noaa-gfs-bdp-pds/gdas.{date}/00/atmos/"  # adjust to your year/month dynamically
file = fs.glob(
    path + "gdas.t00z.sfluxgrbf000.grib2"
)  

grib_file = "data/gdas.t00z.sfluxgrbf000.grib2"
# 3. Open as a single concatenated Dataset
ds_gfs = xr.open_dataset(
    grib_file,
    engine="cfgrib",
    filter_by_keys={'typeOfLevel': 'surface'},
)




/Users/CraigC/Documents/Learning/Mapping_Coding/Projects/can_i_ride_research/.venv/lib/python3.13/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


In [ ]:
ds_gfs

<xarray.Dataset> Size: 547MB
Dimensions:     (latitude: 1536, longitude: 3072)
Coordinates:
    time        datetime64[ns] 8B ...
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
  * latitude    (latitude) float64 12kB 89.91 89.79 89.68 ... -89.79 -89.91
  * longitude   (longitude) float64 25kB 0.0 0.1172 0.2344 ... 359.6 359.8 359.9
    valid_time  datetime64[ns] 8B ...
Data variables: (12/29)
    sp          (latitude, longitude) float32 19MB ...
    orog        (latitude, longitude) float32 19MB ...
    t           (latitude, longitude) float32 19MB ...
    cnwat       (latitude, longitude) float32 19MB ...
    sdwe        (latitude, longitude) float32 19MB ...
    sde         (latitude, longitude) float32 19MB ...
    ...          ...
    sdswrf      (latitude, longitude) float32 19MB ...
    sdlwrf      (latitude, longitude) float32 19MB ...
    suswrf      (latitude, longitude) float32 19MB ...
    sulwrf      (latitude, longitude) float32 19MB ...
    lsm         (latitude, longitude) float32 19MB ...
    siconc      (latitude, longitude) float32 19MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2025-06-03T18:26 GRIB to CDM+CF via cfgrib-0.9.1...